In [1]:
#install pulp module
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 28.7 MB/s eta 0:00:00


In [2]:
# import the neccessary libraries
import pandas as pd
from pulp import  *

In [3]:
# import the dataset into a dataframe
df = pd.read_csv("https://raw.githubusercontent.com/Nishaanthan/People_Analytics/refs/heads/main/datasets/fau_medical_staff.csv", index_col=0)

df = df.fillna(0).map(lambda x: 1 if x == "X" else x)

In [4]:
df.head()

,Shift 1,Shift 2,Shift 3,Avg_Patient_Number
Time Windows,,,,
06:00 - 07:00,1,0,0,4.0
07:00 – 08:00,1,0,0,3.0
08:00 – 09:00,1,0,0,6.0
09:00 – 10:00,1,0,0,8.0
10:00 – 11:00,1,0,0,7.0


In [5]:
# create a matrix to show which shift each time window is associated with
shifts = df.drop(index=["Wage rate per 8h shift (EUR)"], columns="Avg_Patient_Number").values

In [6]:
# read the rest of the data from the the csv file

# number of shifts
shift_num = shifts.shape[1]

# number of time windows
time_windows = shifts.shape[0]

# number of patients measured per time window
avg_patient_num = df["Avg_Patient_Number"].values

# wage rate per shift
wages_per_shift = df.loc["Wage rate per 8h shift (EUR)", :].values.astype(int)

# service level to handle 4 patients
service_level = 4

In [7]:
# Determine the decision variable
# final goal is to find the optimal number of workers for each time slot
num_workers = LpVariable.dicts("num_workers", [s for s in range(shift_num)], lowBound=0, cat="Integer")
print(num_workers)

{0: num_workers_0, 1: num_workers_1, 2: num_workers_2}


In [8]:
# Create problem
# Minimize number of workers/costs paid for employees each day
prob = LpProblem("Optimal_Medical_Assistants_FAU", LpMinimize)

In [9]:
# add the contsraints to the LP problem
# first add to the problem the wage for each shift
prob += lpSum([num_workers[s] * wages_per_shift[s] for s in range(shift_num)]), "Minimize_Wages"

In [10]:
# the average number of customers in each time slot and the service level are also added to the LP problem
for t in range(time_windows):
    for s in range(shift_num):
        if shifts[t, s] == 1:
            prob += num_workers[s] * service_level >= avg_patient_num[t], f"Demand_Constraint_Shift_{s + 1}_Time_Window_{t + 1}"


In [11]:
# solve the problem

prob.solve()
print("Status:", LpStatus[prob.status])

Status: Optimal


In [12]:
for s in range(shift_num):
    print(f"{int(num_workers[s].varValue)} medical assistants are needed for Shift {s + 1}")

3 medical assistants are needed for Shift 1
2 medical assistants are needed for Shift 2
1 medical assistants are needed for Shift 3
